## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

#printing to make sure the above code worked correctly 
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

#print to make sure the above worked 
#application_cat

application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Check the value counts of CLASSIFICATION column
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value based on the counts
cutoff = 1000  # You can adjust this value based on the counts

# Identify values to replace with "Other"
classification_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace values in the dataframe
for cls in classification_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.head(20)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1883
classifications_to_replace = []
for classification, count in classification_counts.items():
  if count < cutoff_value:
    classifications_to_replace.append(classification)

# Look at the list of classifications to replace
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_transformed = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_transformed['IS_SUCCESSFUL'].values
X = application_df_transformed.drop(['IS_SUCCESSFUL'], axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = 44, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\mahan\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           3,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,061 (23.68 KB)

 Trainable params: 6,061 (23.68 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 554us/step - accuracy: 0.7135 - loss: 0.5861
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.7309 - loss: 0.5551
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 611us/step - accuracy: 0.7328 - loss: 0.5498
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step - accuracy: 0.7325 - loss: 0.5506
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 602us/step - accuracy: 0.7320 - loss: 0.5498
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.7266 - loss: 0.5536
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7378 - loss: 0.5452
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7307 - loss: 0.5549
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7328 - loss: 0.5486
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7366 - loss: 0.5476
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 600us/step - accuracy: 0.7324 - loss: 0.5484
Epoch 12/100
804/80

804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step - accuracy: 0.7422 - loss: 0.5334
Epoch 68/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 600us/step - accuracy: 0.7449 - loss: 0.5343
Epoch 69/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 602us/step - accuracy: 0.7396 - loss: 0.5353
Epoch 70/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.7351 - loss: 0.5398
Epoch 71/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - accuracy: 0.7407 - loss: 0.5366
Epoch 72/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.7399 - loss: 0.5383
Epoch 73/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.7410 - loss: 0.5358
Epoch 74/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step - accuracy: 0.7431 - loss: 0.5317
Epoch 75/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 0.7409 - loss: 0.5363
Epoch 76/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 0.7401 - loss: 0.5368
Epoch 77/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.7413 - loss: 0.5359
Epoch 78/100
804/804 ━━

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 776us/step - accuracy: 0.7262 - loss: 0.5546
Loss: 0.5546185970306396, Accuracy: 0.7261807322502136


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")